In [1]:
import cobra
from cobra.test import create_test_model
from cobra.flux_analysis.gapfilling import GapFiller
from medusa.core.ensemble import Ensemble
from medusa.flux_analysis import flux_balance
from medusa.reconstruct.expand.expand import gapfill_to_ensemble
import pandas as pd
import random

In [2]:
def construct_textbook_ensemble():
    # create two identical models and make an ensemble
    model1 = create_test_model("textbook")
    model1.remove_reactions(model1.reactions[1:3])
    model1.id = 'first_textbook'
    model2 = create_test_model("textbook")
    model2.remove_reactions(model2.reactions[4:6])
    model2.id = 'second_textbook'
    textbook_ensemble = Ensemble(model_list=[model1,model2],
                                        base_id='textbook_ensemble')
    return textbook_ensemble

def construct_mixed_ensemble():
    # create 4 models, which have reactions removed and a bound difference.
    model1 = create_test_model("textbook")
    model1.remove_reactions(model1.reactions[1:3])
    model1.id = 'first_textbook'
    model2 = create_test_model("textbook")
    model2.remove_reactions(model2.reactions[4:6])
    model2.id = 'second_textbook'
    model3 = create_test_model("textbook")
    model3.remove_reactions(model3.reactions[5:7])
    model3.id = 'third_textbook'
    model4 = model3.copy()
    model4.id = 'dual_features'
    model4.reactions[1].lower_bound = 0
    model_list = [model1,model2,model3,model4]
    
    mixed_ensemble = Ensemble(list_of_models=model_list,identifier='textbook_ensemble')
    return mixed_ensemble

def break_for_gapfill(model,n_reactions):
    model = model.copy()
    non_boundaries = [rxn for rxn in model.reactions if rxn not in model.boundary]
    reaction_indices = random.sample(list(range(0,len(non_boundaries))),n_reactions)
    universal_model = cobra.core.Model()
    rxns_to_remove = [non_boundaries[rxn] for rxn in reaction_indices]
    rxn_copies = [rxn.copy() for rxn in rxns_to_remove]
    model.remove_reactions(rxns_to_remove)
    universal_model.add_reactions(rxn_copies)
    return(model,universal_model)

In [3]:
testmod = create_test_model("salmonella")
# open all exchange reactions
for rxn in testmod.exchanges:
    rxn.lower_bound = -1
breakmod,universal = break_for_gapfill(testmod,200)

In [4]:
testmod

Name,Salmonella_consensus_build_1
Memory address,0x01070cf7f0
Number of metabolites,1802
Number of reactions,2546
Objective expression,1.0*biomass_iRR1083_metals - 1.0*biomass_iRR1083_metals_reverse_00f08
Compartments,"Cytoplasm, Periplasm, Extracellular"


In [5]:
breakmod

Name,Salmonella_consensus_build_1
Memory address,0x0108de3240
Number of metabolites,1802
Number of reactions,2346
Objective expression,1.0*biomass_iRR1083_metals - 1.0*biomass_iRR1083_metals_reverse_00f08
Compartments,"Cytoplasm, Periplasm, Extracellular"


In [6]:
breakmod.solver='gurobi'

In [7]:
gapfilled_ensemble = gapfill_to_ensemble(breakmod,iterations=20,universal=universal,\
                                       lower_bound=0.5,penalties=None,exchange_reactions=False,\
                                       demand_reactions=False, integer_threshold=1E-8)

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Parameter Method unchanged
   Value: 0  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
finished gap-filling. Constructing ensemble...


TypeError: '<' not supported between instances of 'Reaction' and 'Reaction'

In [ ]:
gapfilled_ensemble

In [ ]:
fill_dem = GapFiller(breakmod,universal=universal,\
                                       lower_bound=0.5,penalties=None,exchange_reactions=False,\
                                       demand_reactions=False, integer_threshold=1E-8)

In [ ]:
sols = fill_dem.fill(iterations=100)

In [ ]:
len(sols[0])

In [ ]:
sol = []
for s in sols:
    sol = sol + s
len(set(sol))

In [ ]:
gapfill_solutions[0]

In [ ]:
sol = []
for s in gapfill_solutions:
    sol = sol + s
len(set(sol))

In [ ]:
for exchange in test_ensemble.base_model.medium.keys():
    medium = test_ensemble.base_model.medium
    old = medium[exchange]
    medium[exchange] = 0
    test_ensemble.base_model.medium = medium
    print(test_ensemble.base_model.medium)
    sol = flux_balance.optimize_ensemble(test_ensemble,'Biomass_Ecoli_core')
    print(sol)
    medium[exchange] = old
    test_ensemble.base_model.medium = medium

In [ ]:
test_ensemble.base_model.medium['EX_glc__D_e']

In [ ]:
model = create_test_model('salmonella')

In [ ]:
model

In [ ]:
# randomly remove reactions from salmonella model and generate copies
def degrade_model(model,model_count,n_reactions):
    list_of_models = []
    for i in range(0,model_count):
        reaction_indices = random.sample(list(range(0,len(model.reactions))),n_reactions)
        model_copy = model.copy()
        rxns_to_remove = [model_copy.reactions[i] for i in reaction_indices]
        model_copy.remove_reactions(rxns_to_remove)
        model_copy.id = model_copy.id + str(i)
        list_of_models.append(model_copy)
    return list_of_models


In [ ]:
nreactions = 20
lists_of_models = {}
for i in [10,20,30,40,50]:
    lists_of_models[i] = degrade_model(model,i,20)
    

In [ ]:
lists_of_models = {}
lists_of_models[200] = degrade_model(model,200,20)

In [ ]:
import time
for model_count in lists_of_models.keys():
    start = time.time()
    salmonella_ensemble = Ensemble(list_of_models=lists_of_models[model_count],
                                        identifier='salmonella_ensemble')
    end = time.time()
    print(model_count,end - start)


In [ ]:
salmonella_ensemble.base_model.objective.expression

In [ ]:
flux_balance.optimize_ensemble(salmonella_ensemble)